In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("https://github.com/woldemarg/ds_tests/blob/master/kaggle_solutions/home_credit_default_risk/data/samples/app_samp.csv.gz?raw=true",
                   index_col=0,
                   compression="gzip")

# desc = pd.read_csv("data/raw/HomeCredit_columns_description.csv",
#                    index_col=0,
#                    quotechar='"')

In [3]:
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
295391,442225,0,Cash loans,F,N,N,0,157500.0,314100.0,17167.5,225000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,Rented apartment,0.010147,-9684,-1056,-3077.0,-2361,NaN,1,1,0,1,0,0,Sales staff,1.0,2,2,SATURDAY,12,0,0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
298766,446119,0,Cash loans,M,Y,Y,0,382500.0,402214.5,31905.0,328500.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.010147,-18901,-2159,-29.0,-2379,25.0,1,1,1,1,0,0,Managers,2.0,2,2,THURSDAY,12,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1443.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0
279614,423933,0,Cash loans,F,N,Y,0,112500.0,913500.0,30321.0,913500.0,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,0.005144,-20982,365243,-7793.0,-4016,NaN,1,0,0,1,0,0,NaN,2.0,2,2,SATURDAY,10,0,0,0,0,0,0,...,0.1702,0.2124,0.0,0.0,reg oper account,block of flats,0.2117,Panel,No,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
303407,451526,1,Cash loans,M,Y,Y,2,202500.0,970380.0,31432.5,810000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.020713,-10242,-724,-2483.0,-2821,11.0,1,1,1,1,1,0,Laborers,4.0,3,3,TUESDAY,13,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,-2225.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0
158531,283788,0,Cash loans,F,N,N,0,222750.0,315000.0,19269.0,315000.0,Unaccompanied,Working,Secondary / secondary special,Widow,House / apartment,0.031329,-22435,-6054,-9661.0,-4914,NaN,1,1,1,1,1,0,Managers,1.0,2,2,TUESDAY,14,0,0,0,0,0,0,...,0.0684,0.0870,0.0,0.0,reg oper account,block of flats,0.0739,Panel,No,0.0,0.0,0.0,0.0,-2193.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0


In [4]:
univ = pd.Series(data=[data[col].nunique()
                       if data[col].dtype == "object"
                       else np.nan
                       for col in data.columns],
                 index=data.columns)

stats = pd.concat([data.dtypes,
                   univ,
                   data.isna().mean().round(4)],
                  axis=1)

stats.columns = ["type", "univ", "pct_nan"]

In [5]:
print(stats)

                               type  univ  pct_nan
SK_ID_CURR                    int64   NaN   0.0000
TARGET                        int64   NaN   0.0000
NAME_CONTRACT_TYPE           object   2.0   0.0000
CODE_GENDER                  object   2.0   0.0000
FLAG_OWN_CAR                 object   2.0   0.0000
...                             ...   ...      ...
AMT_REQ_CREDIT_BUREAU_DAY   float64   NaN   0.1294
AMT_REQ_CREDIT_BUREAU_WEEK  float64   NaN   0.1294
AMT_REQ_CREDIT_BUREAU_MON   float64   NaN   0.1294
AMT_REQ_CREDIT_BUREAU_QRT   float64   NaN   0.1294
AMT_REQ_CREDIT_BUREAU_YEAR  float64   NaN   0.1294

[122 rows x 3 columns]


Первое знакомство с датасетом, статистика по типам, количеству категорий для категориальных переменных, процент пропусков. Последующий отбор предикторов - на основе этой статистики, а также "экспертно-аналитическим" путем с опорой на описание переменных.

На сначала - разделение на обучающую и проверочную выборку.

In [6]:
data.set_index("SK_ID_CURR",
               drop=True,
               inplace=True)

X = data.drop(["TARGET"], axis=1)
y = data["TARGET"]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    train_size=0.8,
                                                    random_state=1234,
                                                    stratify=data["TARGET"])

X_train = X_train.copy()
X_test = X_test.copy()

Часть переменных - частота запросов в Бюро кридатныхисторий и т.п. Здесь пропуски могут означать отстутсиве запросов. Предпологаю, можно безболезненно запонить 0.

In [7]:
cols_to_impute_w_null = ["AMT_GOODS_PRICE",
                         "OBS_30_CNT_SOCIAL_CIRCLE",
                         "DEF_30_CNT_SOCIAL_CIRCLE",
                         "OBS_60_CNT_SOCIAL_CIRCLE",
                         "DEF_60_CNT_SOCIAL_CIRCLE",
                         "OWN_CAR_AGE",
                         "AMT_REQ_CREDIT_BUREAU_HOUR",
                         "AMT_REQ_CREDIT_BUREAU_DAY",
                         "AMT_REQ_CREDIT_BUREAU_WEEK",
                         "AMT_REQ_CREDIT_BUREAU_MON",
                         "AMT_REQ_CREDIT_BUREAU_QRT",
                         "AMT_REQ_CREDIT_BUREAU_YEAR"]

# for col in cols_to_impute_w_null:
#     X_train.loc[:, col + "_was_missing"] = X_train[col].isnull() * 1
#     X_test.loc[:, col + "_was_missing"] = X_test[col].isnull() * 1

X_train.loc[:, cols_to_impute_w_null] = (X_train[cols_to_impute_w_null]
                                         .fillna(0, axis=1))

X_test.loc[:, cols_to_impute_w_null] = (X_test[cols_to_impute_w_null]
                                        .fillna(0, axis=1))

Еще пропуски в малозначащайпеременной. Можно заполнить модой. по факту - мода "Без сопровождения", так что почти то же, что и пропущенное значение.

In [8]:
X_test.loc[:, "NAME_TYPE_SUITE"] = (X_test["NAME_TYPE_SUITE"]
                                    .fillna(X_train["NAME_TYPE_SUITE"]
                                            .mode()[0]))

X_train.loc[:, "NAME_TYPE_SUITE"] = (X_train["NAME_TYPE_SUITE"]
                                     .fillna(X_train["NAME_TYPE_SUITE"]
                                             .mode()[0]))

В переменных, характеризующих жилье очень много хаотичных пропусков. Заполнять их нечем. Принял решение свести весь этот набор переменных к одной, которая бы  характеризовала в общем доступность/полноту информации о жилье (отношение количества заполненных граф к их общему числу)

In [9]:
apartments_cols = data.columns[data.columns
                               .str.endswith(("_AVG", "_MODE", "_MEDI"))]

X_train["APART_DESC_INTEGRITY"] = (1 - X_train[apartments_cols]
                                   .isnull().mean(axis=1))

X_test["APART_DESC_INTEGRITY"] = (1 - X_test[apartments_cols]
                                  .isnull().mean(axis=1))

Бальная оценка клиента из сторонних источников мне показалась важнойпеременной. Всего источника три, по отдельны мклиентом есть информация из разного количества источников. В каждой колонке данные нормализованы, так что независимо от того, в какой шкале источник дает исходную информацию, мы работаем с коэффициентами от 0 до 1. Стало быть их можно перемножить и получить некий интегральный итоговый показатель. Вопрос в том, как привязать количество источников при работе с интегоральным показателем. Весов для каждого источника у нас нет, но можно предположить, что чем больше источников дают информацию о клиенте, тем она достовернее. Т.е. 3 лучше, чем 2, и тем более, 1. Предположил, что можно в качестве весов использовать простую пропорцию отношения количества источников по клиенту к максимально возможному их числу (3).

In [10]:
ext_source_cols = data.columns[data.columns
                               .str.startswith("EXT_")]


def get_ext_score(row):
    if row[ext_source_cols].isnull().sum() == 3:
        return 0
    return (row[ext_source_cols]
            .mean() *
            (1 - row[ext_source_cols]
             .isnull().mean()))


X_train["EXT_SOURCE_INTEGRITY"] = X_train.apply(get_ext_score, axis=1)
X_test["EXT_SOURCE_INTEGRITY"] = X_test.apply(get_ext_score, axis=1)

В переменной "Род деятельности". Тем не менее, предположил, что можно попробовать их заполнить на основе множества других данных о клиенте, обычно ассоциирующихся с профессией (образование, пол, сфера деятельности) Возможно,стоило бы учесть и возраст. Переменная "Род деятельности" нам понадобится для нормализации стажа работы в диапазоне от 0 до 1, т.к. с точки зрения задачи коассификации, нам интересны не только абсолюьныецифры по клиенту, но также и понимание насколько он более стабилен на своей работем относительно других наших клиентов.  

In [11]:
occu_gr_cols = ["NAME_INCOME_TYPE",
                "NAME_EDUCATION_TYPE",
                "CODE_GENDER",
                "ORGANIZATION_TYPE"]

occupations = (X_train
               .groupby(occu_gr_cols)["OCCUPATION_TYPE"]
               .apply(lambda d: d.value_counts(dropna=False).index[0])
               .replace(np.nan, "Missed"))

X_train.reset_index(inplace=True)
X_train.set_index(occu_gr_cols, inplace=True)
X_train.update(occupations)
X_train.reset_index(inplace=True)
X_train.set_index(["SK_ID_CURR"], inplace=True)

X_test.reset_index(inplace=True)
X_test.set_index(occu_gr_cols, inplace=True)
X_test.update(occupations)
X_test.reset_index(inplace=True)
X_test.set_index(["SK_ID_CURR"], inplace=True)

Собственно проводим такую нормализацию стажа работы, используя только что дозаполненную переменную "Род деятельности" длясоздания "профессиональных групп" 

In [12]:
exp_gr_cols = ["ORGANIZATION_TYPE", "OCCUPATION_TYPE"]

exp_gr_range = (X_train
                .groupby(exp_gr_cols)["DAYS_EMPLOYED"]
                .agg(["min", "max"]))

exp_gr_range.columns = ["EXP_GR_MAX", "EXP_GR_MIN"]

X_train = (X_train
           .merge(exp_gr_range,
                  how="left",
                  left_on=exp_gr_cols,
                  right_index=True))

X_test = (X_test
          .merge(exp_gr_range,
                 how="left",
                 left_on=exp_gr_cols,
                 right_index=True))


def normalize_exp(row):
    if row["EXP_GR_MAX"] == row["EXP_GR_MIN"]:
        return 0
    return ((row["DAYS_EMPLOYED"] - row["EXP_GR_MIN"]) /
            (row["EXP_GR_MAX"] - row["EXP_GR_MIN"]))


X_train["DAYS_EMPLOYED_GR_NORM"] = X_train.apply(normalize_exp,
                                                 axis=1)

X_test["DAYS_EMPLOYED_GR_NORM"] = X_test.apply(normalize_exp,
                                               axis=1)

Следующие показатели, мне кажется, должны считаться класическимидля финансовых задачтакого рода. Отношение суммы ежегодных платежей по кредиту к годовому доходу - показатель, своего рода, наличия запаса прочности у клиента.
Оношение сумму покупки к сумме кредита - экспериментальный показатель. Обратил внимание, что есть записи с суммойкредита несколько большепокупки. Тут может быть широкое поле для трактовки. Для банка, например, это может быть как оценка стоимости залоговоа имуществав случаеневозврата кредита. Можно, опять же, трактовать и так: если заказчик берет кредит больше суммы покупки, то часть денег, вероятно, идет на приобретение сопутствующих услуг. И, если, например, клиент почувтсвует фин. сложности с возвратом долга, тоон можжет реализовать товар частично илиполностью, длярасчета с банком, но не услуги.  

In [13]:
X_train = X_train.eval("ANNUITY_RATIO = AMT_ANNUITY / AMT_INCOME_TOTAL")
X_test = X_test.eval("ANNUITY_RATIO = AMT_ANNUITY / AMT_INCOME_TOTAL")

X_train = X_train.eval("GOODS_RATIO = AMT_GOODS_PRICE / AMT_CREDIT")
X_test = X_test.eval("GOODS_RATIO = AMT_GOODS_PRICE / AMT_CREDIT")

Избавляемся от отработанных (по жилью, по внешнейоценке) и "технических" колонок (экстремумы для нормализации стажа), и кодируем категориальныепеременныесредствами pandas.

In [14]:
cols_to_drop = (apartments_cols.tolist() +
                ext_source_cols.tolist() +
                exp_gr_range.columns.tolist())

X_train.drop(cols_to_drop,
             axis=1,
             inplace=True)

X_test.drop(cols_to_drop,
            axis=1,
            inplace=True)

In [15]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

X_train, X_test = X_train.align(X_test, join="left", axis=1)

Поскольку датасет несбалансирован, добавляеммодель в модель веса. Весь анализ проводился на репрезентативном сэмпле (1%) от основной выборки - вот [код](https://github.com/woldemarg/ds_tests/blob/master/kaggle_solutions/home_credit_default_risk/scripts/get_samples.py) 

In [16]:
target = y_train.value_counts()
spw = target[0] / target[1]

xgb_model = XGBClassifier(random_state=1234,
                          objective="binary:logistic",
                          scale_pos_weight=spw,
                          n_jobs=-1)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

print("ROC-AUC on test: {}".format(roc_auc_score(y_test, y_pred)))

ROC-AUC on test: 0.6651590106007067


Выбранная метрика (ROC-AUC) на сэмпле - 0.6.
Если отбросить также и абсолютные величины, для которыхбылирасчитаны относительные (типа суммы товара и пр.) метрика ухудшается.
По построении модели по большей выборке (50% рядов) метрика улушается практически до 0,68 (здесь не показано). И еще немного растет при простейшемподборе гиперпараметров - см. [код](https://github.com/woldemarg/ds_tests/blob/master/kaggle_solutions/home_credit_default_risk/scripts/baseline_pipe.py)
Итоговый результат мродели по всему датасету на тестовых данных - 0,7 (см. [ноутбук](https://www.kaggle.com/volodymyrholomb/xgbmodel-on-base-features))

In [17]:
xgb_fea_imp = (pd.DataFrame(list(xgb_model.get_booster()
                                 .get_fscore().items()),
                            columns=["feature", "importance"])
               .sort_values("importance", ascending=False))


Как видим, большинство предложенных дополнительных предикторов попало в топ-15 наиболее значимых. Дальше можно пробовать успростить модель, отбросив наименне значимые предикторы, и оценить результат. 

In [18]:
xgb_fea_imp.head(15)

,feature,importance
0,EXT_SOURCE_INTEGRITY,64
13,DAYS_EMPLOYED,42
2,DAYS_REGISTRATION,31
11,DAYS_ID_PUBLISH,27
9,AMT_INCOME_TOTAL,25
28,AMT_ANNUITY,23
10,DAYS_LAST_PHONE_CHANGE,22
38,DAYS_BIRTH,21
27,ANNUITY_RATIO,19
12,AMT_CREDIT,17
